---

# 🎯 PHASE 1: QUICK WINS - Foundation Improvements

## 1.1 Advanced Data Augmentation

### ❓ Tại Sao Cần Cải Thiện?

**Vấn đề của mô hình gốc:**
- Chỉ sử dụng augmentation cơ bản (flip, rotate)
- Không tận dụng domain knowledge của medical imaging
- Generalization kém khi gặp variations mới

### 💡 Giải Pháp

Sử dụng **Albumentations** với các augmentation được thiết kế đặc biệt cho X-ray:

1. **CLAHE (Contrast Limited Adaptive Histogram Equalization)**
   - Cải thiện contrast cho ảnh X-ray
   - Giúp highlight các vùng bệnh lý không rõ ràng

2. **ShiftScaleRotate**
   - Mô phỏng các góc chụp khác nhau
   - Robust với positioning variations

3. **GaussNoise & GaussianBlur**
   - Mô phỏng chất lượng ảnh khác nhau
   - Robust với imaging equipment variations

### 📈 Expected Impact
- **+1-2% AUC** improvement
- Better generalization to unseen data
- Reduced overfitting

In [1]:
def get_train_transforms(img_size=224):
    """
    Advanced augmentation pipeline cho training data
    
    Thiết kế dựa trên:
    1. Medical imaging best practices
    2. Empirical studies on chest X-ray augmentation
    3. ImageNet normalization cho transfer learning
    """
    return A.Compose([
        # Resize & crop
        A.Resize(int(img_size * 1.15), int(img_size * 1.15)),
        A.RandomCrop(img_size, img_size),
        
        # Geometric transformations
        A.HorizontalFlip(p=0.5),  # X-ray có thể flip horizontally
        A.ShiftScaleRotate(
            shift_limit=0.1,      # Shift 10% - mô phỏng positioning
            scale_limit=0.15,     # Scale ±15% - mô phỏng khoảng cách chụp
            rotate_limit=15,      # Rotate ±15° - mô phỏng góc chụp
            border_mode=cv2.BORDER_CONSTANT,
            value=0,
            p=0.5
        ),
        
        # Noise & blur - mô phỏng chất lượng thiết bị
        A.OneOf([
            A.GaussNoise(var_limit=(10, 50), p=1.0),
            A.GaussianBlur(blur_limit=(3, 5), p=1.0),
            A.MotionBlur(blur_limit=5, p=1.0),
        ], p=0.3),
        
        # Contrast & brightness - critical for X-ray
        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.2,
            p=0.5
        ),
        
        # CLAHE - Medical imaging specific
        # Cải thiện contrast cục bộ, quan trọng cho phát hiện bệnh lý
        A.CLAHE(
            clip_limit=4.0,
            tile_grid_size=(8, 8),
            p=0.5
        ),
        
        # Optional: Grid distortion (mô phỏng deformation)
        A.GridDistortion(
            num_steps=5,
            distort_limit=0.05,
            p=0.2
        ),
        
        # Normalization - ImageNet stats cho transfer learning
        A.Normalize(
            mean=[0.485, 0.456, 0.406],  # ImageNet mean
            std=[0.229, 0.224, 0.225],   # ImageNet std
        ),
        ToTensorV2(),
    ])

def get_valid_transforms(img_size=224):
    """
    Validation transforms - NO augmentation
    Chỉ resize và normalize
    """
    return A.Compose([
        A.Resize(img_size, img_size),
        A.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        ),
        ToTensorV2(),
    ])

print("✅ Advanced augmentation pipeline created")
print("📋 Training augmentations:")
print("   - Resize & Random Crop")
print("   - Horizontal Flip")
print("   - ShiftScaleRotate")
print("   - Noise & Blur variations")
print("   - Brightness & Contrast")
print("   - CLAHE (Medical-specific)")
print("   - Grid Distortion")

✅ Advanced augmentation pipeline created
📋 Training augmentations:
   - Resize & Random Crop
   - Horizontal Flip
   - ShiftScaleRotate
   - Noise & Blur variations
   - Brightness & Contrast
   - CLAHE (Medical-specific)
   - Grid Distortion


### 🔍 Visualization: So Sánh Augmentation

Hãy xem sự khác biệt giữa augmentation cơ bản và advanced

In [2]:
def visualize_augmentations(image_path, n_samples=6):
    """
    Visualize effect of augmentation pipeline
    """
    # Load image
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Get transforms
    train_transform = get_train_transforms(224)
    
    # Create figure
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Advanced Augmentation Examples', fontsize=16, fontweight='bold')
    
    axes = axes.ravel()
    
    for idx in range(n_samples):
        # Apply augmentation
        augmented = train_transform(image=image)
        aug_image = augmented['image']
        
        # Denormalize for visualization
        mean = torch.tensor([0.485, 0.456, 0.406]).view(3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225]).view(3, 1, 1)
        aug_image = aug_image * std + mean
        aug_image = aug_image.permute(1, 2, 0).numpy()
        aug_image = np.clip(aug_image, 0, 1)
        
        axes[idx].imshow(aug_image)
        axes[idx].set_title(f'Augmented Sample {idx+1}')
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

print("📸 Augmentation visualization function ready")
print("   Use: visualize_augmentations('path/to/xray.png')")

📸 Augmentation visualization function ready
   Use: visualize_augmentations('path/to/xray.png')
